# Analisi dati italiani Covid19

Scarico i dati disponibili nella cartella della protezione civile su Github

In [1]:
library(RCurl)
library(dplyr)
library(magrittr)
library(ggplot2)
x <- getURL("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv")
y <- read.csv(text = x, stringsAsFactors = FALSE)

Warning message:
"package 'RCurl' was built under R version 3.6.2"Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'magrittr' was built under R version 3.6.3"Warning message:
"package 'ggplot2' was built under R version 3.6.3"

In [2]:
head(y)

data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,note_it,note_en
2020-02-24T18:00:00,ITA,13,Abruzzo,42.35122,13.39844,0,0,0,0,0,0,0,0,0,5,NA,NA
2020-02-24T18:00:00,ITA,17,Basilicata,40.63947,15.80515,0,0,0,0,0,0,0,0,0,0,NA,NA
2020-02-24T18:00:00,ITA,4,P.A. Bolzano,46.49933,11.35662,0,0,0,0,0,0,0,0,0,1,NA,NA
2020-02-24T18:00:00,ITA,18,Calabria,38.90598,16.59440,0,0,0,0,0,0,0,0,0,1,NA,NA
2020-02-24T18:00:00,ITA,15,Campania,40.83957,14.25085,0,0,0,0,0,0,0,0,0,10,NA,NA
2020-02-24T18:00:00,ITA,8,Emilia Romagna,44.49437,11.34172,10,2,12,6,18,18,0,0,18,148,NA,NA


Pre-processing dei dati:
1) rimuovere colonne che non servono o ridondanti
2) missing data non esistenti 
3) rimuovere orario dalle date

In [3]:
y$stato <- NULL
y$codice_regione <- NULL
y$lat <- NULL
y$long <- NULL
y$data <- substr(y$data, 1, 10)
y <- mutate(y, data= as.Date(data))

In [4]:
head(y)

data,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,note_it,note_en
2020-02-24,Abruzzo,0,0,0,0,0,0,0,0,0,5,NA,NA
2020-02-24,Basilicata,0,0,0,0,0,0,0,0,0,0,NA,NA
2020-02-24,P.A. Bolzano,0,0,0,0,0,0,0,0,0,1,NA,NA
2020-02-24,Calabria,0,0,0,0,0,0,0,0,0,1,NA,NA
2020-02-24,Campania,0,0,0,0,0,0,0,0,0,10,NA,NA
2020-02-24,Emilia Romagna,10,2,12,6,18,18,0,0,18,148,NA,NA


Andamento regione Puglia per quanto riguarda i positivi dall'inizio del contagio

In [5]:
ypuglia <- y[y$denominazione_regione == 'Puglia', ] 

In [6]:
## Sort by date
ypuglia = ypuglia[order(as.Date(ypuglia$data)),]
ypuglia$data <- format(as.Date(ypuglia$data), "%d/%m")
head(ypuglia)

,data,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,note_it,note_en
14,24/02,Puglia,0,0,0,0,0,0,0,0,0,0,NA,NA
35,25/02,Puglia,0,0,0,0,0,0,0,0,0,0,NA,NA
56,26/02,Puglia,0,0,0,0,0,0,0,0,0,0,NA,NA
77,27/02,Puglia,1,0,1,0,1,1,0,0,1,0,NA,NA
98,28/02,Puglia,1,0,1,2,3,2,0,0,3,242,NA,NA
119,29/02,Puglia,1,0,1,2,3,0,0,0,3,252,NA,NA


## Andamento ospedalizzati

In [7]:
# Stacked Bar Plot with Colors and Legend
#data_selected <- t(as.matrix(subset(ypuglia, select=c("data", "totale_ospedalizzati", "isolamento_domiciliare"))))
#barplot(ypuglia$totale_ospedalizzati)

# Give the chart file a name
png(file = "ospedalizzati_puglia.png", width = 1000, height = 800, res = 200)

# Plot the bar chart 
barplot(ypuglia$totale_ospedalizzati,names.arg=ypuglia$data,xlab="Data",ylab="Tot ospedalizzati",col="blue",
main="Andamento ospedalizzati in Puglia",border="red",ylim=range(pretty(c(0, ypuglia$totale_ospedalizzati))))

# Save the file
dev.off()

png 
  2

## Andamento con distribuzione tra ospedalizzati e isolamento domiciliare

In [8]:
# Create the input vectors.
colors = c("red","blue")
dates <- c(ypuglia$data)
regions <- c("Tot in ospedale","Isolamento domiciliare")

# Create the matrix of the values.
Values <- matrix(c(ypuglia$totale_ospedalizzati,ypuglia$isolamento_domiciliare), nrow = 2, ncol = length(dates), byrow = TRUE)
#Values <- t(as.matrix(subset(ypuglia, select=c("data", "totale_ospedalizzati", "isolamento_domiciliare"))))

# Give the chart file a name
png(file = "Distrib_isolamento_ospedale.png")

# Create the bar chart
barplot(Values, main = "Distribuzione tra isolamento e ospedalizzati in Puglia", names.arg = dates, xlab = "Data", ylab = "Totale positivi", col = colors, )

# Add the legend to the chart
legend("topleft", regions, cex = 1.3, fill = colors)

# Save the file
dev.off()

png 
  2

## Andamento con animazione della Nazione

In [9]:
## Sort by date
y = y[order(as.Date(y$data)),]
#y$data <- format(as.Date(y$data), "%d/%m")
head(y)

data,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,note_it,note_en
2020-02-24,Abruzzo,0,0,0,0,0,0,0,0,0,5,NA,NA
2020-02-24,Basilicata,0,0,0,0,0,0,0,0,0,0,NA,NA
2020-02-24,P.A. Bolzano,0,0,0,0,0,0,0,0,0,1,NA,NA
2020-02-24,Calabria,0,0,0,0,0,0,0,0,0,1,NA,NA
2020-02-24,Campania,0,0,0,0,0,0,0,0,0,10,NA,NA
2020-02-24,Emilia Romagna,10,2,12,6,18,18,0,0,18,148,NA,NA


In [10]:
# libraries:
library(ggplot2)
library(gganimate)
library(hrbrthemes)
library(viridis)
library(extrafont)

# Keep only 3 provinces
y_filtered <- y %>% 
  filter(denominazione_regione %in% c("Lombardia", "Veneto", "Puglia"))

# Plot
plot <- y_filtered %>%
  ggplot( aes(x=data, y=totale_attualmente_positivi, group=denominazione_regione, color=denominazione_regione)) +
    geom_line() +
    geom_point() +
    scale_color_viridis(discrete = TRUE) +
    ggtitle("Totale attualmente positivi in alcune regioni") +
    theme_bw() +
    ylab("Totale positivi") +
    transition_reveal(data, range = seq(as.Date("2020-02-24"), by = "month", length.out = 30))

# Save at gif:
animate(plot, height = 400, width =750, nframes = 300, fps = 30)
anim_save("andamento contagiati.gif")

Warning message:
"package 'gganimate' was built under R version 3.6.3"Warning message:
"package 'hrbrthemes' was built under R version 3.6.3"Warning message:
"package 'viridis' was built under R version 3.6.3"Loading required package: viridisLite
Warning message:
"package 'viridisLite' was built under R version 3.6.3"Warning message:
"package 'extrafont' was built under R version 3.6.2"Registering fonts with R
Warning message in (nframes - 1) * (v - range[1])/full_length:
"longer object length is not a multiple of shorter object length"Warning message in (nframes - 1) * (v - range[1])/full_length:
"longer object length is not a multiple of shorter object length"

# Studio su dati province 

In [34]:
library(RCurl)
library(dplyr)
library(magrittr)
xprov <- getURL("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province.csv")
yprov <- read.csv(text = xprov, stringsAsFactors = FALSE)

In [35]:
head(yprov)

data,stato,codice_regione,denominazione_regione,codice_provincia,denominazione_provincia,sigla_provincia,lat,long,totale_casi,note_it,note_en
2020-02-24T18:00:00,ITA,13,Abruzzo,69,Chieti,CH,42.35103,14.16755,0,,
2020-02-24T18:00:00,ITA,13,Abruzzo,66,L'Aquila,AQ,42.35122,13.39844,0,,
2020-02-24T18:00:00,ITA,13,Abruzzo,68,Pescara,PE,42.46458,14.21365,0,,
2020-02-24T18:00:00,ITA,13,Abruzzo,67,Teramo,TE,42.65892,13.70440,0,,
2020-02-24T18:00:00,ITA,13,Abruzzo,979,In fase di definizione/aggiornamento,,0.00000,0.00000,0,,
2020-02-24T18:00:00,ITA,17,Basilicata,77,Matera,MT,40.66751,16.59792,0,,


Pre-processing dei dati:
1) rimuovere colonne che non servono o ridondanti
2) missing data non esistenti 
3) rimuovere orario dalle date

In [36]:
yprov$stato <- NULL
yprov$codice_provincia <- NULL
yprov$sigla_provincia <- NULL
yprov$lat <- NULL
yprov$long <- NULL
yprov$data <- substr(yprov$data, 1, 10)
yprov <- mutate(yprov, data= as.Date(data))

In [37]:
head(yprov)

data,codice_regione,denominazione_regione,denominazione_provincia,totale_casi,note_it,note_en
2020-02-24,13,Abruzzo,Chieti,0,,
2020-02-24,13,Abruzzo,L'Aquila,0,,
2020-02-24,13,Abruzzo,Pescara,0,,
2020-02-24,13,Abruzzo,Teramo,0,,
2020-02-24,13,Abruzzo,In fase di definizione/aggiornamento,0,,
2020-02-24,17,Basilicata,Matera,0,,


Andamento regione a scelta per quanto riguarda i positivi dall'inizio del contagio

In [38]:
regione <- 'Puglia'
yprov <- yprov[yprov$denominazione_regione == regione, ] 
## Sort by date
yprov_datashort = yprov[order(as.Date(yprov$data), yprov$denominazione_provincia),]
yprov_datashort$data <- format(as.Date(yprov$data), "%d/%m")
head(yprov_datashort)

,data,codice_regione,denominazione_regione,denominazione_provincia,totale_casi,note_it,note_en
93,24/02,19,Sicilia,Agrigento,0,,
94,24/02,19,Sicilia,Caltanissetta,0,,
95,24/02,19,Sicilia,Catania,0,,
96,24/02,19,Sicilia,Enna,0,,
102,24/02,19,Sicilia,In fase di definizione/aggiornamento,0,,
97,24/02,19,Sicilia,Messina,0,,


In [39]:
provincia <- 'Trapani'
yprov_filtered <- yprov_datashort[yprov_datashort$denominazione_provincia == provincia, ]
head(yprov_filtered)

,data,codice_regione,denominazione_regione,denominazione_provincia,totale_casi,note_it,note_en
101,24/02,19,Sicilia,Trapani,0,,
229,25/02,19,Sicilia,Trapani,0,,
357,26/02,19,Sicilia,Trapani,0,,
485,27/02,19,Sicilia,Trapani,0,,
613,28/02,19,Sicilia,Trapani,0,,
741,29/02,19,Sicilia,Trapani,0,,


## Andamento dei casi in provincia a scelta nella regione di prima

In [40]:
# Stacked Bar Plot with Colors and Legend
#data_selected <- t(as.matrix(subset(ypuglia, select=c("data", "totale_ospedalizzati", "isolamento_domiciliare"))))
#barplot(ypuglia$totale_ospedalizzati)

provincia <- 'Foggia'
yprov_filtered <- yprov_datashort[yprov_datashort$denominazione_provincia == provincia, ]

# Give the chart file a name
png(file = "totcasi_foggia.png", width = 1500, height = 800, res = 200)

# Plot the bar chart 
barplot(yprov_filtered$totale_casi,names.arg=yprov_filtered$data,xlab="Data",ylab="Tot casi",col="blue",
main="Andamento casi Covid19 in provincia di Trapani",border="red",ylim=range(pretty(c(0, yprov_filtered$totale_casi))))

# Save the file
dev.off()

png 
  2

## Distribuzione dei casi nelle varie provincie della regione scelta prima

In [41]:
# Remove rows with in fase di definizione o aggiornamento
yprov_datashort = yprov_datashort[!yprov_datashort$denominazione_provincia == "In fase di definizione/aggiornamento", ]

# Create the input vectors.
colors = rainbow(length(unique(yprov_datashort$denominazione_provincia)))
dates <- unique(yprov_datashort$data)
regions <- unique(yprov_datashort$denominazione_provincia)

# Create the matrix of the values.
Values <- matrix(c(yprov_datashort$totale_casi), nrow = length(unique(yprov_datashort$denominazione_provincia)), ncol = length(dates), byrow = FALSE)

# Give the chart file a name
png(file = paste("Distrib_casi_", paste(regione, ".png", sep=""), sep=""), width = 2000, height = 800, res = 200)

# Create the bar chart
barplot(Values, main = paste("Distribuzione casi Covid19 in ", regione), names.arg = dates, xlab = "Data", ylab = "Totale casi", col = colors)

# Add the legend to the chart
legend("topleft", regions, cex = 1.3, fill = colors)

# Save the file
dev.off()

png 
  2

In [42]:
yprov_datashort

,data,codice_regione,denominazione_regione,denominazione_provincia,totale_casi,note_it,note_en
93,24/02,19,Sicilia,Agrigento,0,,
94,24/02,19,Sicilia,Caltanissetta,0,,
95,24/02,19,Sicilia,Catania,0,,
96,24/02,19,Sicilia,Enna,0,,
97,24/02,19,Sicilia,Messina,0,,
98,24/02,19,Sicilia,Palermo,0,,
99,24/02,19,Sicilia,Ragusa,0,,
100,24/02,19,Sicilia,Siracusa,0,,
101,24/02,19,Sicilia,Trapani,0,,
221,25/02,19,Sicilia,Agrigento,0,,


## Animazione totale casi in regione scelta

In [43]:
## Sort by date
yprov = yprov[order(as.Date(yprov$data), yprov$denominazione_provincia),]
yprov = yprov[!yprov$denominazione_provincia == "In fase di definizione/aggiornamento", ]

# libraries:
library(ggplot2)
library(gganimate)
library(hrbrthemes)
library(viridis)
library(extrafont)

# Plot
plot <- yprov %>%
  ggplot( aes(x=data, y=totale_casi, group=denominazione_provincia, color=denominazione_provincia)) +
    geom_line() +
    geom_point() +
    scale_color_viridis(discrete = TRUE) +
    ggtitle(paste("Totale casi nelle province della regione ", regione, sep = "")) +
    theme_bw() +
    ylab("Totale casi") +
    transition_reveal(data, range = seq(as.Date("2020-02-24"), by = "month", length.out = 30))

# Save at gif:
animate(plot, height = 400, width =750, nframes = 300, fps = 30)
anim_save(paste(paste("andamento contagiati in ", regione, sep = ""), ".gif", sep = ""))

Warning message in (nframes - 1) * (v - range[1])/full_length:
"longer object length is not a multiple of shorter object length"Warning message in (nframes - 1) * (v - range[1])/full_length:
"longer object length is not a multiple of shorter object length"

# Studio nuovi contagiati

## Livello nazionale

In [44]:
# Select only data and nuovi_attualmente_positivi
ynuovipos = subset(y, select=c("data", "nuovi_attualmente_positivi"))

# Group by data and sum on positivi
ynuovipos_data = aggregate(nuovi_attualmente_positivi ~ data, ynuovipos, sum)

# Plot the trend
library(ggplot2)
theme_set(theme_dark())

png(file = "nuovi_contagi_nazionale.png", width = 1900, height = 800, res=250)

# Allow Default X Axis Labels
ggplot(ynuovipos_data, aes(x=data)) + 
  geom_line(aes(y=nuovi_attualmente_positivi)) + 
  labs(title="Andamento nuovi positivi in Italia", 
       subtitle="Giorno per giorno i nuovi positivi", 
       caption="Source: Protezione civile", 
       y="Nuovi positivi",
       x="")

dev.off()

png 
  2

## Livello regionale

In [47]:
# Select only data and nuovi_attualmente_positivi
regione = 'Puglia'
ynuovipos_reg = subset(y[y$denominazione_regione == regione,], select=c("data", "nuovi_attualmente_positivi"))

# Plot the trend
library(ggplot2)
theme_set(theme_dark())

png(file = paste(paste("nuovi_contagi_", regione, sep=""), ".png", sep=""), width = 1900, height = 800, res=300)

# Allow Default X Axis Labels
ggplot(ynuovipos_reg, aes(x=data)) + 
  geom_line(aes(y=nuovi_attualmente_positivi)) + 
  labs(title=paste("Andamento nuovi positivi in ", regione, sep=""), 
       subtitle="Giorno per giorno i nuovi positivi", 
       caption="Source: Protezione civile", 
       y="Nuovi positivi",
       x="")

dev.off()

png 
  2